<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/micro_seq_bn_helpfulness_classification_multi_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from time import time
start_time = time()

In [ ]:
!pip install transformers[torch]
!pip install datasets
!pip install scikit-learn
!pip install adapters

In [ ]:
from sklearn.metrics import accuracy_score,  f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [ ]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_MICRO_helpfulness_dataset"
dataset = load_dataset(dataset_name)
dataset

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#Label Settings
id2label = {0: "unhelpful", 1: "helpful"}
label2id = {"unhelpful": 0, "helpful": 1}

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=adapter_hub_name + "_model",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.0001, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10, # NEED TO TRAIN FOR REAL TEST
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=1, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    load_best_model_at_end=True, # Set to false - we want the last trained model like the paper
    metric_for_best_model="f1_macro", # Is default
    # torch_compile=True,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
    seed=0,
)

In [ ]:
from transformers import EarlyStoppingCallback

# EarlyStoppingCallback with patience
early_stopping = EarlyStoppingCallback(early_stopping_patience=3) # from paper

In [ ]:
from adapters import AdapterTrainer
load_name = f"BigTMiami/micro_helpfulness_tapt_pretrain_seq_bn_adapter"
import gc


In [ ]:
all_train_results = []
all_eval_results = []

In [ ]:
for seed in range(2):
  print(f"Seed: {seed}")
  print("=========================================")
  training_args.seed = seed

  # Load Pretrained adapter without head
  loaded_adapter_name = model.load_adapter(load_name, with_head=False)

  # Add head for classification modeling
  model.add_classification_head(
      loaded_adapter_name,
      num_labels=2,
      id2label={ 0: "unhelpful", 1: "helpful"})

  # Set the adapter to be used for training
  model.train_adapter(loaded_adapter_name)

  adapter_hub_name = f"micro_seq_bn_helpfulness_classification_adapter_seed_{seed}"

  summary = model.adapter_summary()
  print(summary)

  trainer = AdapterTrainer(
      model=model,
      args=training_args,
      train_dataset=dataset["train"],
      eval_dataset=dataset["dev"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      callbacks=[early_stopping]
  )

  results = trainer.train()
  print(results)
  all_train_results.append(results)

  eval_results = trainer.evaluate(dataset["test"])
  print(eval_results)
  all_eval_results.append(eval_results)

  model.push_adapter_to_hub(
      adapter_hub_name,
      loaded_adapter_name,
      datasets_tag=dataset_name
  )
  print(f"Pushed {adapter_hub_name}")

  # Delete the seed adapter
  model.set_active_adapters(None)
  model.delete_adapter(loaded_adapter_name)

  gc.collect()

In [ ]:
for result in all_train_results:
  print(result)

for result in all_eval_results:
  print(result)

In [ ]:
end_time = time()
total_time = end_time - start_time
print(f"Total Script Runtime: {total_time / 60:.0f}  minutes OR {total_time:.0f}  seconds")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()
